In [174]:
import pandas as pd
import numpy as np
import datetime as dt
import pandas.stats.moments as st
import matplotlib.pyplot as plt
from pandas_datareader.data import Options
import py_vollib
from py_vollib.black_scholes_merton.implied_volatility import *
from py_vollib.black_scholes_merton.greeks.analytical import *
import sched, time
from collections import OrderedDict
%matplotlib inline


In [172]:
# Function historical data from alpha advantage
def historical_data(ticker, day_number = 252, rolling_window = 20, outsize = 'full'):
    alphavantage_link = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={0}&apikey=5HZEUI5AFJB06BUK&datatype=csv&outputsize={1}'.format(ticker, outsize)
    stockframe = pd.read_csv(alphavantage_link, index_col = 0).sort_index()[['open', 'close']]
    stockframe['daily_ret'] = np.log(stockframe['close']/stockframe['close'].shift(1))
    stockframe['intra_ret'] = np.log(stockframe['close']/stockframe['open'])
    stockframe['ovrnt_ret'] = np.log(stockframe['open']/stockframe['close'].shift(1))
    stockframe['daily_vol'] = stockframe.daily_ret.rolling(window=rolling_window,center=False).std()
    stockframe['intra_vol'] = stockframe.intra_ret.rolling(window=rolling_window,center=False).std()
    stockframe['ovrnt_vol'] = stockframe.ovrnt_ret.rolling(window=rolling_window,center=False).std()
    stockframe['daily_ann'] = stockframe.daily_vol*np.sqrt(252)
    stockframe['intra_ann'] = stockframe.intra_vol*np.sqrt((24/6.5)*252)
    stockframe['ovrnt_ann'] = stockframe.ovrnt_vol*np.sqrt((24/17.5)*252)
    stockframe['oc_diff'] = stockframe.close - stockframe.open
    stockframe['daily_dollar_vol'] = stockframe.daily_vol*stockframe.close.shift(1)
    stockframe['daily_dollar_std'] = np.abs(stockframe.oc_diff/stockframe.daily_dollar_vol)

    return stockframe

def all_options(ticker):
    tape = Options(ticker, 'yahoo')
    data = tape.get_all_data().reset_index()
    
    data['Moneyness'] = np.abs(data['Strike'] - data['Underlying_Price'])/data['Underlying_Price']
    
    data['DTE'] = (data['Expiry'] - dt.datetime.today()).dt.days
    data = data[['Strike', 'DTE', 'Type', 'IV', 'Vol','Open_Int', 'Moneyness', 'Root', 'Underlying_Price',
                 'Last','Bid','Ask','Expiry']]
    data['Mid'] = (data['Ask'] - data['Bid'])/2 + data['Bid']
    
    year = 365
    strikes = data['Strike'].values
    time_to_expirations = data['DTE'].values
    ivs = data['IV'].values
    underlying = data['Underlying_Price'].values[0]
    types = data['Type'].values

    # Make sure nothing thows up
    assert len(strikes) == len(time_to_expirations)

    sigmas = data['IV']
    deltas = []
    gammas = []
    thetas = []
    vegas = []
    for sigma, strike, time_to_expiration, flag in zip(sigmas, strikes, time_to_expirations, types):

        # Constants
        S = underlying
        K = strike
        t = time_to_expiration/float(year)
        r = 0.005 / 100
        q = 0 / 100

        try:
            delta = py_vollib.black_scholes_merton.greeks.analytical.delta(flag[0], S, K, t, r, sigma, q)
            deltas.append(delta)
        except:
            delta = 0.0
            deltas.append(delta)

        try:
            gamma = py_vollib.black_scholes_merton.greeks.analytical.gamma(flag[0], S, K, t, r, sigma, q)
            gammas.append(gamma)
        except:
            gamma = 0.0
            gammas.append(gamma)

        try:
            theta = py_vollib.black_scholes_merton.greeks.analytical.theta(flag[0], S, K, t, r, sigma, q)
            thetas.append(theta)
        except:
            theta = 0.0
            thetas.append(theta)

        try:
            vega = py_vollib.black_scholes_merton.greeks.analytical.vega(flag[0], S, K, t, r, sigma, q)
            vegas.append(vega)
        except:
            vega = 0.0
            vegas.append(vega)

    data['Delta'] = deltas
    data['Gamma'] = gammas
    data['Theta'] = thetas
    data['Vega'] = vegas

    return data.reset_index()[data.columns]

def maturities(date):
    
    # Calculate today, but note that since we are adjusting for lookback bias, we need to change the current date to one day prior
    today = date
    curr_month = today.month
    curr_year = today.year
    
    # Finding Prev Third Wed
    curr_eigth_day = dt.date(curr_year,curr_month,7)
    curr_second_day = dt.date(curr_year,curr_month,3).weekday()
    curr_third_fri = curr_eigth_day - dt.timedelta(curr_second_day) + dt.timedelta(14)
    last_third_wed = curr_third_fri - dt.timedelta(30)
    
    # Finding Next Third Wed
    if curr_month == 12:
        next_month = 2
        next_year = curr_year + 1
    elif curr_month == 11:
        next_month = 1
        next_year = curr_year + 1
    else:
        next_month = curr_month + 2
        next_year = curr_year
    next_eigth_day = dt.date(next_year,next_month,7)
    next_second_day = dt.date(next_year,next_month,3).weekday()
    next_third_fri = next_eigth_day - dt.timedelta(next_second_day) + dt.timedelta(14)
    next_third_wed = next_third_fri - dt.timedelta(30)
    
    # Finding Cur Third Wed
    if curr_month == 12:
        next_month = 1
        next_year = curr_year + 1
    else:
        next_month = curr_month + 1
        next_year = curr_year
    next_eigth_day = dt.date(next_year,next_month,7)
    next_second_day = dt.date(next_year,next_month,3).weekday()
    next_third_fri = next_eigth_day - dt.timedelta(next_second_day) + dt.timedelta(14)
    curr_third_wed = next_third_fri - dt.timedelta(30)
    
    # Finding Term: When current date is after expiry, should be 100% of spot/f1
    if today < curr_third_wed:
        dte = curr_third_wed - today
        term = curr_third_wed - last_third_wed
    else:
        dte = next_third_wed - today
        term = next_third_wed - curr_third_wed
    # print (float(dte.days)/term.days)
    front_weight = float(dte.days)/term.days
    back_weight = 1 - front_weight
    return [front_weight, back_weight]

def earnings_condor(tick, max_gap, dte_thresh, money_thresh):
    chain = all_options(tick)
    chain = chain[chain['DTE'] <= dte_thresh]
    chain = chain.reset_index()[chain.columns]
    chain = chain[chain['Moneyness'] <= money_thresh]
    chain_puts = chain[(chain['Type'] == 'put') & (chain['Strike'] < chain['Underlying_Price'].values[0])]
    chain_calls = chain[(chain['Type'] == 'call') & (chain['Strike'] > chain['Underlying_Price'].values[0])]


    put_spread_prem = []
    put_spread_delta = []
    put_spread_gamma = []
    put_spread_vega = []
    put_spread_theta = []
    put_spread_short_strike = []
    put_spread_long_strike = []
    put_spread_max_loss = []
    for idx, row in chain_puts.sort_values('Strike', ascending = False).iterrows():
        curr_short_strike = row.Strike
        curr_short_prem = row.Bid
        curr_short_delta = row.Delta
        curr_short_gamma = row.Gamma
        curr_short_vega = row.Vega
        curr_short_theta = row.Theta

        temp_longs = chain_puts[(chain_puts['Strike'] < curr_short_strike) &
                                (chain_puts['Strike'] >= curr_short_strike - max_gap)]

        for temp_idx, temp_row in temp_longs.iterrows():
            curr_long_strike = temp_row.Strike
            curr_long_prem = temp_row.Ask
            curr_long_delta = temp_row.Delta
            curr_long_gamma = temp_row.Gamma
            curr_long_vega = temp_row.Vega
            curr_long_theta = temp_row.Theta

            curr_spread_prem = curr_short_prem - curr_long_prem
            curr_spread_delta = -curr_short_delta + curr_long_delta
            curr_spread_gamma = -curr_short_gamma + curr_long_gamma
            curr_spread_vega = -curr_short_vega + curr_long_vega
            curr_spread_theta = -curr_short_theta + curr_long_theta
            curr_spread_maxloss = (curr_short_strike - curr_long_strike - curr_spread_prem)*100

            put_spread_prem.append(curr_spread_prem)
            put_spread_delta.append(curr_spread_delta)
            put_spread_gamma.append(curr_spread_gamma)
            put_spread_vega.append(curr_spread_vega)
            put_spread_theta.append(curr_spread_theta)
            put_spread_short_strike.append(curr_short_strike)
            put_spread_long_strike.append(curr_long_strike)
            put_spread_max_loss.append(curr_spread_maxloss)

    put_spreads_df = pd.DataFrame(OrderedDict({'put Combo': range(len(put_spread_prem)),
                                               'Short Put Strike': put_spread_short_strike,
                                               'Long Put Strike': put_spread_long_strike,
                                               'put Spread Premium': put_spread_prem,
                                               'put Spread Maxloss': put_spread_max_loss,
                                               'put Spread Delta': put_spread_delta,
                                               'put Spread Gamma': put_spread_gamma,
                                               'put Spread Vega': put_spread_vega,
                                               'put Spread Theta': put_spread_theta}),
                                  index = range(len(put_spread_prem)))

    call_spread_prem = []
    call_spread_delta = []
    call_spread_gamma = []
    call_spread_vega = []
    call_spread_theta = []
    call_spread_short_strike = []
    call_spread_long_strike = []
    call_spread_max_loss = []
    for idx, row in chain_calls.sort_values('Strike', ascending = True).iterrows():
        curr_short_strike = row.Strike
        curr_short_prem = row.Bid
        curr_short_delta = row.Delta
        curr_short_gamma = row.Gamma
        curr_short_vega = row.Vega
        curr_short_theta = row.Theta

        temp_longs = chain_calls[(chain_calls['Strike'] > curr_short_strike) &
                                (chain_calls['Strike'] <= curr_short_strike + max_gap)]

        for temp_idx, temp_row in temp_longs.iterrows():
            curr_long_strike = temp_row.Strike
            curr_long_prem = temp_row.Ask
            curr_long_delta = temp_row.Delta
            curr_long_gamma = temp_row.Gamma
            curr_long_vega = temp_row.Vega
            curr_long_theta = temp_row.Theta

            curr_spread_prem = curr_short_prem - curr_long_prem
            curr_spread_delta = -curr_short_delta + curr_long_delta
            curr_spread_gamma = -curr_short_gamma + curr_long_gamma
            curr_spread_vega = -curr_short_vega + curr_long_vega
            curr_spread_theta = -curr_short_theta + curr_long_theta
            curr_spread_maxloss = -(curr_short_strike - curr_long_strike + curr_spread_prem)*100

            call_spread_prem.append(curr_spread_prem)
            call_spread_delta.append(curr_spread_delta)
            call_spread_gamma.append(curr_spread_gamma)
            call_spread_vega.append(curr_spread_vega)
            call_spread_theta.append(curr_spread_theta)
            call_spread_short_strike.append(curr_short_strike)
            call_spread_long_strike.append(curr_long_strike)
            call_spread_max_loss.append(curr_spread_maxloss)

    call_spreads_df = pd.DataFrame(OrderedDict({'call Combo': range(len(call_spread_prem)),
                                               'Short call Strike': call_spread_short_strike,
                                               'Long call Strike': call_spread_long_strike,
                                               'call Spread Premium': call_spread_prem,
                                               'call Spread Maxloss': call_spread_max_loss,
                                               'call Spread Delta': call_spread_delta,
                                               'call Spread Gamma': call_spread_gamma,
                                               'call Spread Vega': call_spread_vega,
                                               'call Spread Theta': call_spread_theta}),
                                  index = range(len(call_spread_prem)))

    put_combos = []
    call_combos = []
    condor_prems = []
    condor_maxloss = []
    condor_delta = []
    condor_gamma = []
    condor_vega = []
    condor_theta = []
    put_short = []
    put_long = []
    call_short = []
    call_long = []

    for idxc, rowc in call_spreads_df.iterrows():
        for idxp, rowp in put_spreads_df.iterrows():
            p_s = put_spreads_df[put_spreads_df['put Combo'] == rowp['put Combo']]['Short Put Strike'].values[0]
            p_l = put_spreads_df[put_spreads_df['put Combo'] == rowp['put Combo']]['Long Put Strike'].values[0]
            c_s = call_spreads_df[call_spreads_df['call Combo'] == rowc['call Combo']]['Short call Strike'].values[0]
            c_l = call_spreads_df[call_spreads_df['call Combo'] == rowc['call Combo']]['Long call Strike'].values[0]

            put_short.append(p_s)
            put_long.append(p_l)
            call_short.append(c_s)
            call_long.append(c_l)

            curr_prem = round(rowp['put Spread Premium'] + rowc['call Spread Premium'],2)

            condor_prems.append(curr_prem)
            condor_maxloss.append(100*(max(p_s - p_l, c_l - c_s) - curr_prem))
            condor_delta.append(rowp['put Spread Delta'] + rowc['call Spread Delta'])
            condor_gamma.append(rowp['put Spread Gamma'] + rowc['call Spread Gamma'])
            condor_vega.append(rowp['put Spread Vega'] + rowc['call Spread Vega'])
            condor_theta.append(rowp['put Spread Theta'] + rowc['call Spread Theta'])

    condors_df = pd.DataFrame(OrderedDict({'P Short Strike': put_short,
                                           'P Long Strike': put_long,
                                           'C Short Strike': call_short,
                                           'C Long Strike': call_long,
                                           'Premium': condor_prems,
                                           'Maxloss': condor_maxloss,
                                           'Delta': condor_delta,
                                           'Gamma': condor_gamma,
                                           'Vega': condor_vega,
                                           'Theta': condor_theta}),
                                  index = range(len(condor_prems)))
    condors_df['RiskRewardRatio'] = round((100*condors_df['Premium'])/condors_df['Maxloss'],2)
    put_spreads_df['RiskRewardRatio'] = round((100*put_spreads_df['put Spread Premium'])/put_spreads_df['put Spread Maxloss'],2)
    call_spreads_df['RiskRewardRatio'] = round((100*call_spreads_df['call Spread Premium'])/call_spreads_df['call Spread Maxloss'],2)
    condors_df['Underlying Price'] = chain['Underlying_Price'].values[0]
    
    return condors_df, put_spreads_df, call_spreads_df

In [166]:
start_time = time.time()

spx_options = all_options('^SPX')

vix_options = all_options('^VIX')


spx_daily = historical_data('SPX')

vix_daily = pd.read_csv('https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=VIX&apikey=5HZEUI5AFJB06BUK&datatype=csv&outputsize=full', index_col = 0)

daily_df = np.round(vix_daily[['open','close']].join(spx_daily,how = 'inner', lsuffix='_vix').sort_index(),3)
daily_df.columns = ['VIX Open', 'VIX Close','SPX Open','SPX Close','Daily Return','Intraday Return',
                    'Overnight Return','Daily Vol','Intraday Vol','Overnight Vol','Daily Annual Vol',
                    'Intraday Annual Vol','Overnight Annual Vol',
                    'Open-Close Difference','Daily Dollar Vol','Daily Dollar Std']

vf_df = pd.read_csv('http://173.212.203.121/noko.csv', index_col = 0)[['F1','F2','F3']]
vf_df.index = pd.to_datetime(vf_df.index)

vix_df = vix_daily[['close']].join(vf_df, how = 'inner').sort_index()
vix_df.columns = ['VIX','F1','F2','F3']

contango_ratio = []

vix_df.index = pd.to_datetime(vix_df.index)
for i, row in vix_df.iterrows():
    weights = maturities(i.date())
    curr_ratio = weights[0]*(row.VIX/row.F1) + weights[1]*(row.F1/row.F2)
    contango_ratio.append(round(curr_ratio,3))

vix_df['Contango'] = contango_ratio

print("--- %s seconds ---" % (time.time() - start_time))

C:\Users\Fang\Anaconda3\lib\site-packages\py_vollib\ref_python\black_scholes_merton\__init__.py:87: RuntimeWarning: divide by zero encountered in double_scalars
  return numerator / denominator
C:\Users\Fang\Anaconda3\lib\site-packages\py_vollib\black_scholes_merton\greeks\analytical.py:210: RuntimeWarning: invalid value encountered in double_scalars
  return numerator / denominator
C:\Users\Fang\Anaconda3\lib\site-packages\py_vollib\black_scholes_merton\greeks\analytical.py:153: RuntimeWarning: invalid value encountered in double_scalars
  first_term = (S * numpy.exp(-q*t) * pdf(D1) * sigma) / (2 * numpy.sqrt(t))


--- 20.18545126914978 seconds ---


In [150]:
start_time = time.time()


spx_intraday_link = 'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=SPX&interval=1min&apikey=5HZEUI5AFJB06BUK&datatype=csv'#&outputsize=full'
spx_intraday = pd.read_csv(spx_intraday_link, index_col = 0)[['open','high','low','close']]

vix_intraday_link = 'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=VIX&interval=1min&apikey=5HZEUI5AFJB06BUK&datatype=csv'#&outputsize=full'
vix_intraday = pd.read_csv(vix_intraday_link, index_col = 0)[['open','high','low','close']]

intraday_df = np.round(vix_intraday.join(spx_intraday, how='inner', lsuffix='_vix', rsuffix='_spx'),2).sort_index()
intraday_df.columns = ['VIX Open','VIX High','VIX Low', 'VIX Close', 'SPX Open', 'SPX High', 'SPX Low', 'SPX Close']

rolling_window = 20

intraday_vol = intraday_df[['SPX Close']]
intraday_vol['Log Return'] = np.log(intraday_vol['SPX Close']/intraday_vol['SPX Close'].shift(1))
intraday_vol['Return Std'] = intraday_vol['Log Return'].rolling(window=rolling_window,center=False).std()
intraday_vol['Dollar Std'] = intraday_vol['Return Std']*intraday_vol['SPX Close']
intraday_vol['Dollar Std Move'] = (intraday_vol['SPX Close'] - intraday_vol['SPX Close'].shift(1))/intraday_vol['Dollar Std'].shift(1)
intraday_df['Dollar Std Move'] = intraday_vol['Dollar Std Move']

print("--- %s seconds ---" % (time.time() - start_time))

--- 1.724799633026123 seconds ---


C:\Users\Fang\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Users\Fang\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Fang\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/st

In [175]:
dte_thresh = 0
money_thresh = 0.02
tick = '^SPX'
max_gap = 20

spx_puts = spx_options[(spx_options['DTE'] <= dte_thresh) &
                       (spx_options['Type'] == 'put') &
                       (spx_options['Moneyness'] <= money_thresh) &
                       (spx_options['Strike'] <= spx_options['Underlying_Price'])]

spx_calls = spx_options[(spx_options['DTE'] <= dte_thresh) &
                       (spx_options['Type'] == 'call') &
                       (spx_options['Moneyness'] <= money_thresh) &
                       (spx_options['Strike'] >= spx_options['Underlying_Price'])]

spx_condor, spx_puts, spx_calls = earnings_condor(tick, max_gap, dte_thresh, money_thresh)

C:\Users\Fang\Anaconda3\lib\site-packages\py_vollib\ref_python\black_scholes_merton\__init__.py:87: RuntimeWarning: divide by zero encountered in double_scalars
  return numerator / denominator
C:\Users\Fang\Anaconda3\lib\site-packages\py_vollib\black_scholes_merton\greeks\analytical.py:210: RuntimeWarning: invalid value encountered in double_scalars
  return numerator / denominator
C:\Users\Fang\Anaconda3\lib\site-packages\py_vollib\black_scholes_merton\greeks\analytical.py:153: RuntimeWarning: invalid value encountered in double_scalars
  first_term = (S * numpy.exp(-q*t) * pdf(D1) * sigma) / (2 * numpy.sqrt(t))


In [181]:
spx_condor[(spx_condor['Premium'] >= 0.5) &
           (spx_condor['C Short Strike'] >= 2830) &
           (spx_condor['P Short Strike'] <= 2800)].sort_values('RiskRewardRatio',ascending = False)

P Short Strike  P Long Strike  C Short Strike  C Long Strike  Premium  \
287          2800.0         2795.0          2830.0         2835.0     0.80   
291          2795.0         2790.0          2830.0         2835.0     0.65   
320          2800.0         2790.0          2830.0         2840.0     1.30   
295          2790.0         2785.0          2830.0         2835.0     0.60   
286          2800.0         2790.0          2830.0         2835.0     1.10   
324          2795.0         2785.0          2830.0         2840.0     1.10   
319          2800.0         2785.0          2830.0         2840.0     1.55   
353          2800.0         2785.0          2830.0         2845.0     1.65   
302          2780.0         2775.0          2830.0         2835.0     0.50   
299          2785.0         2780.0          2830.0         2835.0     0.50   
321          2800.0         2795.0          2830.0         2840.0     1.00   
357          2795.0         2780.0          2830.0         2845.0     1.35   
386          2800.0         2780.0          2830.0         2850.0     1.80   
328          2790.0         2780.0          2830.0         2840.0     0.95   
285          2800.0         2785.0          2830.0         2835.0     1.35   
352          2800.0         2780.0          2830.0         2845.0     1.80   
290          2795.0         2785.0          2830.0         2835.0     0.90   
354          2800.0         2790.0          2830.0         2845.0     1.40   
323          2795.0         2780.0          2830.0         2840.0     1.25   
387          2800.0         2785.0          2830.0         2850.0     1.65   
318          2800.0         2780.0          2830.0         2840.0     1.70   
358          2795.0         2785.0          2830.0         2845.0     1.20   
329          2790.0         2785.0          2830.0         2840.0     0.80   
361          2790.0         2775.0          2830.0         2845.0     1.20   
456          2800.0         2790.0          2835.0         2845.0     0.85   
332          2785.0         2775.0          2830.0         2840.0     0.85   
325          2795.0         2790.0          2830.0         2840.0     0.85   
355          2800.0         2795.0          2830.0         2845.0     1.10   
335          2780.0         2770.0          2830.0         2840.0     0.75   
356          2795.0         2775.0          2830.0         2845.0     1.50   
..              ...            ...             ...            ...      ...   
462          2790.0         2770.0          2835.0         2845.0     0.70   
429          2790.0         2775.0          2835.0         2840.0     0.55   
594          2795.0         2775.0          2840.0         2850.0     0.80   
833          2795.0         2780.0          2850.0         2855.0     0.60   
662          2795.0         2775.0          2840.0         2860.0     0.85   
532          2790.0         2780.0          2835.0         2855.0     0.55   
534          2785.0         2765.0          2835.0         2855.0     0.50   
632          2790.0         2770.0          2840.0         2855.0     0.60   
535          2785.0         2770.0          2835.0         2855.0     0.50   
599          2790.0         2775.0          2840.0         2850.0     0.50   
598          2790.0         2770.0          2840.0         2850.0     0.55   
836          2790.0         2770.0          2850.0         2855.0     0.50   
664          2795.0         2785.0          2840.0         2860.0     0.55   
666          2790.0         2770.0          2840.0         2860.0     0.60   
564          2790.0         2770.0          2840.0         2845.0     0.55   
870          2790.0         2770.0          2850.0         2860.0     0.50   
802          2790.0         2770.0          2845.0         2865.0     0.50   
799          2795.0         2780.0          2845.0         2865.0     0.60   
796          2800.0         2790.0          2845.0         2865.0     0.65   
768          2790.0        